In [ ]:
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification --force
!unzip jigsaw-unintended-bias-in-toxicity-classification.zip


Saving kaggle.json to kaggle.json
100% 720M/723M [00:08<00:00, 32.0MB/s]
100% 723M/723M [00:08<00:00, 90.3MB/s]
Archive:  jigsaw-unintended-bias-in-toxicity-classification.zip
  inflating: all_data.csv            
  inflating: identity_individual_annotations.csv  
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test_private_expanded.csv  
  inflating: test_public_expanded.csv  
  inflating: toxicity_individual_annotations.csv  
  inflating: train.csv               


In [ ]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [ ]:
df_train = df.sample(n=500000, random_state=42, ignore_index=True)
df_train

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,593336,0.166667,What a breathe of fresh air to have someone wh...,0.0,0.0,0.0,0.166667,0.0,NaN,NaN,...,151356,approved,0,0,0,4,0,0.0,0,6
1,756192,0.600000,Your jewish friends were the ones who told you...,0.2,0.0,0.6,0.400000,0.0,0.0,0.0,...,158493,approved,0,0,0,0,0,0.0,6,10
2,5407051,0.000000,Possible collusion by Trump and his affiliates...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,343435,approved,0,0,0,1,0,0.0,0,4
3,5808132,0.000000,Exactly. We need a % of GDP spending cap at t...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,368584,approved,0,0,0,7,0,0.0,0,4
4,557013,0.000000,"By your own comment, even if some of them vote...",0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,149754,approved,0,0,0,1,0,0.0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,6075322,0.000000,You're only half right. They don't have to pub...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,384516,approved,0,0,0,1,0,0.0,0,4
499996,848386,0.000000,for the record and historically challenged:\n\...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,162179,approved,1,0,0,1,0,0.0,0,4
499997,943272,0.000000,Know your adversary.\n\nTell them fur is a gre...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,165814,approved,2,0,0,18,5,0.0,4,4
499998,5338625,0.000000,Let's see now. The Trudeau Government adds an ...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,339470,approved,0,0,0,4,1,0.0,0,4


In [ ]:
import numpy as np

# Make sure all comment_text values are strings
df_train['comment_text'] = df_train['comment_text'].astype(str)

# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# Convert taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, 1, 0)

def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

train = convert_dataframe_to_bool(df_train)

In [ ]:
# train_size = 0.8  # Percentage of data for training
# num_samples = len(train)

# # Calculate the number of samples for training and testing
# num_train_samples = int(train_size * num_samples)
# num_test_samples = num_samples - num_train_samples

# # Manually split the data
# X_train, X_test = train[:num_train_samples], train[num_train_samples:]
# y_train, y_test = train['target'][:num_train_samples], train['target'][num_train_samples:]

In [ ]:
# X_train.shape, X_test.shape

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import preprocessing
from tqdm import tqdm

vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(train['comment_text'])
# train['embed'] = X_tfidf.todense()

In [ ]:
train_size = 0.8  # Percentage of data for training
num_samples = len(train)

# Calculate the number of samples for training and testing
num_train_samples = int(train_size * num_samples)
num_test_samples = num_samples - num_train_samples

# Manually split the data
X_train, X_test = X_tfidf[:num_train_samples], X_tfidf[num_train_samples:]
y_train, y_test = train['target'][:num_train_samples], train['target'][num_train_samples:]

X_train_df, X_test_df = train[:num_train_samples], train[num_train_samples:]

In [ ]:
X_test.shape, X_test_df.shape

((100000, 164838), (100000, 45))

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(train, train['target'], test_size=0.2, random_state=42)

In [ ]:
# X_tfidf_train_dense = X_tfidf_train.todense()
# X_tfidf_train_dense.shape

In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
X_train_resampled.shape, y_train_resampled.shape

((735594, 164838), (735594,))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# Train a Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_resampled, y_train_resampled)

# clf = RandomForestClassifier(max_depth=10, random_state=0, n_estimators=600)
# clf.fit(X_train_resampled, y_train_resampled)

MultinomialNB()

In [ ]:
y_pred = nb_classifier.predict(X_test)

In [ ]:
np.unique(y_pred)

array([0, 1])

In [ ]:
y_pred.shape

(100000,)

In [ ]:
MODEL_NAME = 'random_forest'
X_test_df[MODEL_NAME] = y_pred

<ipython-input-130-d9c25ae0dd43>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_df[MODEL_NAME] = y_pred


In [ ]:
X_test_df

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,random_forest
400000,5039357,0,and another tax loving left winger never think...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,2,0,0.0,0,4,1
400001,5260493,0,Mr. Trump is like a 10-year old who has a secr...,0.0,0.0,0.0,0.166667,0.0,NaN,NaN,...,approved,0,0,0,7,2,0.0,0,6,1
400002,922510,0,"Wynne ""says"" a lot of things...Tory was just t...",0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,approved,6,0,0,5,0,0.0,0,4,1
400003,374369,0,the A to B to C of the story is there...D is m...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,0,0,0.0,0,4,0
400004,5976836,0,You are correct it was the NUMMI plant in Cali...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,0,0,0.0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,6075322,0,You're only half right. They don't have to pub...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,1,0,0.0,0,4,0
499996,848386,0,for the record and historically challenged:\n\...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,approved,1,0,0,1,0,0.0,0,4,0
499997,943272,0,Know your adversary.\n\nTell them fur is a gre...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,approved,2,0,0,18,5,0.0,4,4,0
499998,5338625,0,Let's see now. The Trudeau Government adds an ...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,...,approved,0,0,0,4,1,0.0,0,4,0


In [ ]:
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'

In [ ]:
from sklearn import metrics

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive


def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])


def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    # examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    examples = pd.concat([subgroup_negative_examples, non_subgroup_positive_examples], ignore_index=True)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    # examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    examples = pd.concat([subgroup_positive_examples, non_subgroup_negative_examples], ignore_index=True)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [ ]:
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# Convert taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)

def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target']:
        convert_to_bool(bool_df, col)
    for col in identity_columns:
      convert_to_bool(bool_df, col)
    return bool_df

X_test_new = convert_dataframe_to_bool(X_test_df)
X_test_new['male']

400000    False
400001    False
400002    False
400003    False
400004    False
          ...  
499995    False
499996    False
499997    False
499998    False
499999    False
Name: male, Length: 100000, dtype: bool

In [ ]:
bias_metrics_df = compute_bias_metrics_for_model(X_test_new, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
7,white,1445,0.668540,0.617251,0.839683
2,homosexual_gay_or_lesbian,632,0.675468,0.649215,0.815384
5,muslim,1221,0.684901,0.658454,0.816031
4,jewish,431,0.700540,0.724060,0.766028
6,black,808,0.718706,0.663457,0.843312
0,male,2524,0.739932,0.720636,0.809110
1,female,2917,0.743482,0.744372,0.789510
3,christian,2258,0.759797,0.799374,0.750408
8,psychiatric_or_mental_illness,264,0.834201,0.753915,0.868744


In [ ]:
def calculate_overall_auc(dataset, model_name):
    true_labels = dataset[TOXICITY_COLUMN]
    predicted_labels = dataset[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

get_final_metric(bias_metrics_df, calculate_overall_auc(X_test_new, MODEL_NAME))

0.7504770049291689